# Setup

The following cells include the essential importation of libraries, as well as helper functions that are used throughout the notebook.

In [1]:
import os, gc, random

import pandas as pd
import math
import numpy as np

import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    classification_report,
)
from sklearn.preprocessing import MinMaxScaler as MMS

import tensorflow as tf
from tensorflow.python.client import device_lib
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Input, Model

from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Flatten,
    Input,
    Add,
    Activation,
    ZeroPadding2D,
    GlobalAveragePooling2D,
)

from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
    GlobalMaxPooling2D,
    LeakyReLU,
)

from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint,
)

from tensorflow.keras.initializers import (
    glorot_uniform,
    he_uniform,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import (
    preprocess_input,
    decode_predictions,
    preprocess_input,
)

from tensorflow.keras.models import Model, load_model
from tensorflow.keras import utils
from tensorflow.keras.utils import get_file, plot_model, to_categorical, model_to_dot
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50V2, MobileNetV2, VGG16
import tensorflow.keras.backend as K

import cv2
from PIL import Image
import glob
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib.patches import Circle
import scipy as sp

import tensorflow as tf
import numpy as np
from tf_explain.core.grad_cam import GradCAM

import seaborn as sns

from tensorflow.keras.applications import ResNet50V2
from sklearn.metrics import roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from datasets import Dataset
from transformers import ViTImageProcessor
from torchvision import transforms
import torch
import numpy as np
from evaluate import load
from transformers import ViTForImageClassification
from transformers import TrainingArguments, Trainer
from transformers import set_seed

2025-08-10 18:25:30.881802: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-10 18:25:30.881832: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-10 18:25:30.881862: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-10 18:25:30.888199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: /home/isaac/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: ncclCommRegister

This is a garbage collection function to clear as much memory as possible, and prevent the system from crashing.

In [ ]:
def reset_keras():
    tensorflow.keras.backend.clear_session
    gc.collect()

The function is called first to set the stage for the memory to be occupied as much as possible by the following tasks.

In [ ]:
reset_keras()

# Data Manipulation

This section focuses on loading, and preparing data for training, and testing from TFRecord files. A random seed is firstly set so that results are reproducibile.

In [ ]:
SEED = 1234
random.seed(SEED)
tf.config.experimental.enable_op_determinism()
tf.random.set_seed(SEED)

Two empty lists are initialised for images, and labels.

In [ ]:
images, labels = [], []

A dictionary is defined that specifies the structure of TFRecord data.

In [ ]:
features = {
    "label": tf.io.FixedLenFeature([], tf.int64),
    "label_normal": tf.io.FixedLenFeature([], tf.int64),
    "image": tf.io.FixedLenFeature([], tf.string),
}

A function is defined to process the TFRecord files, and store the results into the previously defined ```images```, and ```labels``` lists.

In [ ]:
def read_file(files):
    mydata = (
        tf.data.TFRecordDataset(files, num_parallel_reads=4)
        .shuffle(buffer_size=10000)
        .cache()
    )
    mydata = mydata.map(
        lambda x: tf.io.parse_example(x, features), num_parallel_calls=4
    )
    for image_features in mydata:
        image = tf.io.decode_raw(image_features["image"], tf.uint8)
        image = tf.reshape(image, [299, 299])
        image = np.asarray(image)
        images.append(image)
        labels.append(image_features["label"].numpy())

The garbage collection function is invoked before the files are parsed, to avoid any memory issues.

In [ ]:
reset_keras()

The target TFRecord files are defined in a list, prior to being processed.

In [ ]:
files = [
    "./data/training10_0/training10_0.tfrecords",
    "./data/training10_1/training10_1.tfrecords",
    "./data/training10_2/training10_2.tfrecords",
    "./data/training10_3/training10_3.tfrecords",
    "./data/training10_4/training10_4.tfrecords",
]

The parsing function is called on the previously defined list.

In [ ]:
for file in files:
    read_file(file)

## Preparing Data for Training

This section will encompass cells that process the already loaded training data to make it more understandable to the CNN later in the notebook. Each image is first saved as a ```.jpg``` file in the ```train``` directory with enumerated filenames. The list of labels is saved as a CSV file within the same directory.

In [ ]:
x = 0
for image in images:
    tmp = "./train/"
    tmp += str(x)
    tmp += ".jpg"
    plt.imsave(tmp, image)
    x += 1
np.savetxt("./train/labels.csv", labels, delimiter=",")

The labels are loaded, and converted to boolean format, ideal for the binary classification task the CNN is to perform.

In [ ]:
y_train = np.loadtxt("./train/labels.csv", delimiter=",").astype("int")
y_train = np.where(y_train == 0, 0, y_train)
y_train = np.where(y_train != 0, 1, y_train)
print(np.bincount(y_train) / len(y_train))
y_train = y_train.astype("str")

The list of ```.jpg``` files from the ```train``` directory is retrieving using ```glob```, with the numeric part of the filename extracted, and storing in the ```mysplit``` list for further sorting.

In [ ]:
myf = glob.glob("./train/*.jpg")
mysplit = []

for file in myf:
    filename = file.split("/")[-1]
    number = int(filename.split(".")[0])
    mysplit.append(number)

A Pandas DataFrame is defined for training, with columns to store the image paths, and binary label, all sorted by their numeric value.

In [ ]:
d = {"file": myf, "number": mysplit}
traindf = pd.DataFrame(d)
traindf = traindf.sort_values(by=["number"])
traindf["label"] = y_train

## Preparing Data for Testing

This section applies methods similar to those seen within the previous section, but more suited towards preparing testing data. The following cell begins by loading test images, and labels from NumPy files, and concatenates them to form ```x_test``` for images, and ```y_test``` for labels. The ```y_test``` array is then converted to boolean format as was done for training labels.

In [ ]:
x_test = np.concatenate(
    (
        np.load("./data/cv10_data/cv10_data.npy"),
        np.load("./data/test10_data/test10_data.npy"),
    )
)
y_test = np.concatenate(
    (np.load("./data/cv10_labels.npy"), np.load("./data/test10_labels.npy"))
)
y_test = np.where(y_test == 0, 0, y_test)
y_test = np.where(y_test != 0, 1, y_test)

The testing data is then saved as ```.jpg``` files within the ```test``` directory with enumerated filenames.

In [ ]:
x = 0
for i in range(x_test.shape[0]):
    tmp = "./test/"
    tmp += str(x)
    tmp += ".jpg"
    plt.imsave(tmp, x_test[i, :, :, 0])
    x += 1
np.savetxt("./test/y_test.csv", y_test, delimiter=",")

The test labels are then loaded from the CSV file, and casted as strings.

In [ ]:
y_test = np.loadtxt("./test/y_test.csv", delimiter=",").astype("int")
print(np.bincount(y_test) / len(y_test))
y_test = y_test.astype("str")

Similar to the previous section, the filenames are retrieved as a list of ```.jpg``` files using ```glob```, with the numeric part of the filename extracted via ```glob```.

In [ ]:
myf2 = glob.glob("./test/*.jpg")
mysplit = []

for file in myf2:
    filename = file.split("/")[-1]
    number = int(filename.split(".")[0])
    mysplit.append(number)

A Pandas DataFrame is created with this information.

In [ ]:
d = {"file": myf2, "number": mysplit}
testdf = pd.DataFrame(d)
testdf = testdf.sort_values(by=["number"])
testdf["label"] = y_test

## Visualisation

The dataset is visualised within this section, with the garbage collection function being called first to alleviate the system's memory as much as possible.

In [ ]:
reset_keras()

In [ ]:
fig, ax = plt.subplots(1, 9, figsize=[18, 3])
fig2, ax2 = plt.subplots(1, 9, figsize=[18, 3])
fig.suptitle("Positives Highlighted in Red/Orange", fontsize=16)

for i in range(9):
    tmp = plt.imread(traindf.iloc[i, 0])
    tmp = tmp[:, :, 0]
    tmp2 = y_train[i].astype("int")
    ax[i].imshow(tmp) if tmp2 == 0 else ax[i].imshow(tmp, cmap="hot")
    fig.subplots_adjust(wspace=0, hspace=0)

for i in range(9):
    tmp = plt.imread(testdf.iloc[i, 0])
    tmp = tmp[:, :, 0]
    tmp2 = y_test[i].astype("int")
    ax2[i].imshow(tmp) if tmp2 == 0 else ax2[i].imshow(tmp, "hot")
    fig2.subplots_adjust(wspace=0, hspace=0)

## Data Generators

The data processed in the previous steps is finally encapsulated into generators to feed it into the model during training, and testing. An ```ImageDataGenerator``` object is first defined to preprocess images by rescaling their pixel values. Each pixel is normalised into the range [0, 255] by being divided by 255. This is known to improve the model's training efficiency, and performance.

In [ ]:
random.seed(1234)

In [ ]:
datagen = ImageDataGenerator(rescale=1.0 / 255)

The training generator yields batches of preprocessed training images, and their corresponding labels from the ```traindf``` DataFrame. This feeds the model during training.

In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=None,
    x_col="file",
    y_col="label",
    class_mode="binary",
    color_mode="rgb",
    target_size=(299, 299),
    shuffle=True,
    batch_size=8,
    subset="training",
)

A similar generator is made for the testing set.

In [ ]:
test_generator = datagen.flow_from_dataframe(
    testdf,
    x_col="file",
    y_col="label",
    class_mode="binary",
    color_mode="rgb",
    target_size=(299, 299),
    batch_size=8,
    shuffle=False,
)

# ViT Model

The ViT model is trained slightly differently, owing to the fact that it utilised a PyTorch backend rather than TensorFlow through the HuggingFace ```transformers``` library. A base model is selected, and fine-tuned on the same dataset as the other models are. A seed is first set for the model training to be deterministic, and stay the same across multiple runs.

In [ ]:
set_seed(42)

Firstly, the dataset is converted into a HuggingFace dataset.

In [ ]:
train_dataset = Dataset.from_pandas(traindf[['file', 'label']])
test_dataset = Dataset.from_pandas(testdf[['file', 'label']])

A ViT image processor is then used to load the pre-trained transformer.

In [ ]:
processor = ViTImageProcessor.from_pretrained('facebook/deit-tiny-patch16-224')

Data augmentation is performed on the training, and testing sets in order to help the model generalise better.

In [ ]:
augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
])

def transform(example_batch):
    images = [augment(Image.open(f).convert("RGB")) for f in example_batch["file"]]
    inputs = processor(images, return_tensors="pt")
    inputs["labels"] = [int(lbl) for lbl in example_batch["label"]]
    return inputs

prepared_train_ds = train_dataset.with_transform(transform)
prepared_test_ds = test_dataset.with_transform(transform)

A collator is defined to produce batches of data.

In [ ]:
def collate_fn(batch):
    return {
        "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
        "labels": torch.tensor([x["labels"] for x in batch])
    }

A metric computing function is also defined.

In [ ]:
metric = load("accuracy")
def compute_metrics(p):
    return metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids
    )

The model is then defined for training.

In [ ]:
labels_list = ["Negative", "Positive"]
model = ViTForImageClassification.from_pretrained(
    'facebook/deit-tiny-patch16-224',
    num_labels=len(labels_list),
    id2label={str(i): c for i, c in enumerate(labels_list)},
    label2id={c: str(i) for i, c in enumerate(labels_list)},
    ignore_mismatched_sizes=True
)

The trainer, with all the parameters for training the model, is defined, including the previously defined functions.

In [ ]:
training_args = TrainingArguments(
    output_dir="./models/vit-base-breast",
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    dataloader_num_workers=4,
    num_train_epochs=5,
    fp16=True,
    save_steps=100,
    gradient_accumulation_steps=4,
    eval_steps=100,
    logging_steps=10,
    learning_rate=5e-5,
    warmup_steps=500,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_train_ds,
    eval_dataset=prepared_test_ds,
    tokenizer=processor,
)

Training, and evaluation commences.

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(prepared_test_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

## Evaluating the Models

In [ ]:
vit_model_tokenizer = ViTImageProcessor.from_pretrained("./models/vit-base-breast")
vit_model = ViTForImageClassification.from_pretrained("./models/vit-base-breast")

In [ ]:
reset_keras()

This section evaluates the previously trained models using standard metrics like accuracy, and loss on the training, and validation sets. In the following cell, these metrics are extracted into variables for later use from the performance of the model across different epochs during training.

Predictions are made on the test set are made in order to generate a final classification report, and confusion matrix.

In [ ]:
test_generator.reset()
vit_predictions = trainer.predict(prepared_test_ds)

The previously-made predictions, and actual values are gathered, and fed into ```scikit-learn``` to generate the confusion matrix.

In [ ]:
vit_predictions = np.argmax(vit_predictions.predictions, axis=1)
vit_act=y_test.astype('int')
vit_cm = confusion_matrix(vit_act, vit_predictions)

The results are then plotted using ```seaborn```.

In [ ]:
plt.figure(figsize=(6, 6))
sns.heatmap(
    vit_cm, annot=True, fmt="d", cmap="Blues", xticklabels=target, yticklabels=target
)
plt.title("Confusion Matrix - ViT")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

The same values are used to generate the final classification report.

In [ ]:
vit_report_dict = classification_report(vit_act, vit_predictions, target_names=["Negative", "Positive"], output_dict=True)
vit_df = pd.DataFrame(vit_report_dict).transpose()
vit_df.style.background_gradient(cmap="Blues").format("{:.2f}")

In [ ]:
fpr, tpr, _ = roc_curve(vit_act, vit_predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - ViT')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# Model Explanation

This final section touches upon XAI or explainable AI, by implementing _Grad-CAM_ through the ```tf-explain``` module. This provides a visual indication of the parts of the image that influenced the classification made of the model, explaining the results achieved. This is done for the first ten images within the testing set, but can easily be done for any other image as well. The following function accepts a path to an image, the model used, and it's final layer, and returns the explained version of the image, and the classification result.

In [ ]:
vgg16_base = VGG16(weights="imagenet", include_top=True)

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle("Grad-CAM Visualizations for Test Images 1-10 - VGG16", fontsize=18)

for i in range(1, 11):
    img = tf.keras.preprocessing.image.load_img(f"./test/{i}.jpg", target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    input_array = np.expand_dims(img_array, axis=0)

    preds = vgg16_base.predict(input_array)
    predicted_class = np.argmax(preds[0])

    explainer = GradCAM()
    grid = explainer.explain(
        validation_data=(input_array, None),
        model=vgg16_base,
        class_index=predicted_class,
        layer_name="block5_conv3",
    )

    row = (i - 1) // 5
    col = (i - 1) % 5
    ax = axes[row, col]
    ax.imshow(grid)
    ax.axis("off")
    ax.set_title(f"Img {i} - Class {predicted_class}")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In the above plots, warmer colours indicate higher influence on the model's predicted class, and cooler colours the opposite.